In [1]:
import bqplot
import ytree
import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import *
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw)


In [2]:
a = ytree.load("rockstar_halos/out_0.list")
# fn = a.save_arbor()
# a = ytree.load(fn)

Performance will be improved by saving this arbor with "save_arbor" and reloading:
	>>> a = ytree.load("rockstar_halos/out_0.list")
	>>> fn = a.save_arbor()
	>>> a = ytree.load(fn)


In [3]:
df_trees = {}
print(a.field_list)
for tree in a:
    scale = tree["prog", "scale_factor"]
    assert(scale.size == np.unique(scale).size)
    Mvir = tree["prog", "Mvir_all"]
    vrms = tree["prog", "Vrms"]
    rvir = tree["prog", "Rvir"]
    ti = str(tree)
    d = pd.DataFrame({('Mvir'): Mvir, ('Vrms'): vrms, ('Rvir'): rvir}, index = scale)
    df_trees[ti] = d
df = pd.concat(df_trees)
df.index.names = ["node", "scale"]

#df.columns.names = "node", "field"

['ID', 'DescID', 'Mvir', 'Vmax', 'Vrms', 'Rvir', 'Rs', 'Np', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'JX', 'JY', 'JZ', 'Spin', 'rs_klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'spin_bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'scale_factor', 'uid', 'desc_uid']


In [4]:
df2 = df.unstack(0)

In [5]:
ind = np.argsort(-df2["Mvir"].loc[1.0])

mvir = df2["Mvir"].iloc[-1]
vrms = df2["Vrms"].iloc[-1]
# rvir = df2["Rvir"].iloc[-1]
data = np.array([vrms,mvir])
df2["Mvir"].columns[ind]
mvir = mvir.reindex(df2["Mvir"].columns[ind], axis=1)
vrms = vrms.reindex(df2["Vrms"].columns[ind], axis=1)
# rvir = rvir.reindex(df2["Mvir"].columns[ind], axis=1)

In [26]:
x_scale = bqplot.LinearScale()
y_scale = bqplot.LogScale()

sc_x = LinearScale()
sc_y = LinearScale()
sc_xax = Axis(label=('Vrms'), scale=sc_x, orientation='horizontal')
sc_yax = Axis(label=('Mvir'), scale=sc_y, orientation='vertical')
sc_x2 = LinearScale()
sc_y2 = LinearScale()
sc_xax2 = Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal')
sc_yax2 = Axis(label=('Mvir'), scale=sc_y2, orientation='vertical')


scatter = bqplot.Scatter(x = vrms, y = mvir, 
                        scales = {'x': sc_x, 'y': sc_y}, colors = ['blue'])
scatter2 = bqplot.Scatter(x = [], y = [],
                        scales = {'x': sc_x2, 'y': sc_y2}, colors = ['red'], visible = False)

db_scat_brush = HTML(value='[]')

chosen_y_axis = mvir

def brush_callback(change):    
    highlight = bselect.selected
    db_scat_brush.value = str(highlight)
    if change.new is not None:
        try:
            x_condition = (vrms>=float(highlight[0][0])) & (vrms<=float(highlight[1][0]))
            y_condition = (mvir>=float(highlight[0][1])) & (mvir<=float(highlight[1][1]))
            x_y_cond = x_condition & y_condition
            scatter2.x = vrms[x_y_cond]
            scatter2.y = chosen_y_axis[x_y_cond]
            scatter2.visible = True
        except IndexError:
            ;

def dropdown_callback(change):
    chosen_y_axis = df2[change].iloc[-1]
    chosen_y_axis.reindex(df2[change].columns[ind], axis=1)
    scatter2.y = chosen_y_axis

y_options = ['Spin', 'Rvir', 'Mvir']

y_selector = widgets.Dropdown(
    handler=dropdown_callback,
    options=y_options,
    value='Mvir',
    description='Y Axis:',
    )
            
bselect = bqplot.interacts.BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatter])

bselect.observe(brush_callback, names=['selected'])

fig_scat_brush1 = Figure(marks=[scatter], axes=[sc_xax, sc_yax], title='Mvir vs Vrms',
                        interaction=bselect)
fig_scat_brush2 = Figure(marks=[scatter2], axes=[sc_xax2, sc_yax2], title='Mvir vs Vrms')

left_box = VBox([db_scat_brush, fig_scat_brush1])
right_box = VBox([fig_scat_brush2,y_selector])

HBox([left_box, right_box])

HBox(children=(VBox(children=(HTML(value='[]'), Figure(axes=[Axis(label='Vrms', scale=LinearScale()), Axis(label='Mvir', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushSelector(marks=[Scatter(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([           nan,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
                  nan,   968.47998047,   441.38000488,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   316.98999023,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   304.86999512,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   321.48999023,   332.07000732,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   370.67999268,
          79.79000092,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         343.55999756,   402.42001343,   344.1000061 ,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   326.85998535,   968.47998047,
         968.47998047,   968.47998047,            nan,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,            nan,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,  1156.84997559,   968.47998047,   968.47998047,
         186.91000366,   968.47998047,   968.47998047,   968.47998047,
                  nan,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         968.47998047,   968.47998047,     0.        ,   968.47998047,
         968.47998047,   968.47998047,   968.47998047,   968.47998047,
         135.33000183,   968.47998047,   968.47998047,   968.47998047,
         248.94000244,   968.47998047,            nan,   345.1499939 ,
         968.47998047,   968.47998047,   968.47998047,    97.62000275,
         968.47998047,   968.47998047,   968.47998047,  1041.26000977,
         968.47998047,   968.47998047,   968.47998047,   195.16999817,
         968.47998047,   968.47998047,   968.47998047,   260.3500061 ,
         968.47998047,   333.89001465,   171.44000244,   235.97000122,
         403.44000244,   160.25      ,   378.5       ,   288.04000854,
         968.47998047,   299.04000854,   968.47998047,   968.47998047,
         968.47998047,   347.67999268,   968.47998047,   968.47998047,
         968.47998047,            nan,   968.47998047,   968.47998047,
                  nan,   968.47998047,   968.47998047,   152.36999512,
         968.47998047,   968.47998047,            nan,   968.47998047,
         280.5       ,            nan,   285.27999878,   429.67999268,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,   277.07998657,
         968.47998047,   457.76000977,            nan,   968.47998047,
                  nan,   143.02999878,   322.20999146,   493.05999756,
         304.69000244,   403.07998657,   308.30999756,            nan,
         361.70001221,          

In [19]:
redshift_slider = widgets.FloatSlider(value=0.0,min=0.0,max=6.0,step=.1,description='Redshift',
                                      disabled=False,continuous_update=False,orientation='horizontal',
                                      readout=True,readout_format='.1f')

display(redshift_slider)

def on_value_change(change):
    print(change['new'])

redshift_slider.observe(on_value_change, names='value')


# traitlets = widgets.Dropdown( options=['1', '2', '3'], value='2', description='Traits:', disabled=False)

FloatSlider(value=0.0, continuous_update=False, description='Redshift', max=6.0, readout_format='.1f')

In [23]:
help(widgets.Dropdown())

Help on Dropdown in module ipywidgets.widgets.widget_selection object:

class Dropdown(_Selection)
 |  Allows you to select a single item from a dropdown.
 |  
 |  Parameters
 |  ----------
 |  options: list or dict
 |      The options for the dropdown. This can either be a list of values, e.g.
 |      ``['Galileo', 'Brahe', 'Hubble']`` or ``[0, 1, 2]``, a list of
 |      (label, value) pairs, e.g.
 |      ``[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)]``,
 |      or a dictionary mapping the labels to the values, e.g. ``{'Galileo': 0,
 |      'Brahe': 1, 'Hubble': 2}``. The labels are the strings that will be
 |      displayed in the UI, representing the actual Python choices, and should
 |      be unique. If this is a dictionary, the order in which they are
 |      displayed is not guaranteed.
 |  
 |  index: int
 |      The index of the current selection.
 |  
 |  value: any
 |      The value of the current selection. When programmatically setting the
 |      value, a reverse lookup 